In [2]:
%pip install transformers datasets sentencepiece accelerate jieba scikit-learn

You should consider upgrading via the '/Users/danilkladnitsky/.pyenv/versions/3.10.4/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
ROOT = "../../"

BASE_MODEL_NAME="uer/gpt2-chinese-cluecorpussmall"
SAVE_MODEL_PATH= ROOT + "models/"
DATASET_PATH = ROOT + "train_datasets/txt/"

In [4]:
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [1]:
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer, GPT2LMHeadModel,
    DataCollatorForLanguageModeling, Trainer, TrainingArguments
)
from sklearn.model_selection import train_test_split

class ChineseGPTTrainer:
    def __init__(
        self,
        model_name: str,
        dataset_path: str,
        output_dir: str,
        test_size: float = 0.1,
        max_length: int = 64,
        batch_size: int = 2,
        gradient_accumulation_steps: int = 4,
        epochs: int = 3,
        learning_rate: float = 2e-5,
    ):
        self.model_name = model_name
        self.dataset_path = dataset_path
        self.output_dir = output_dir
        self.test_size = test_size
        self.max_length = max_length
        self.batch_size = batch_size
        self.gradient_accumulation_steps = gradient_accumulation_steps
        self.epochs = epochs
        self.learning_rate = learning_rate

        # Tokenizer and model
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        self.model = GPT2LMHeadModel.from_pretrained(model_name)

        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
            self.model.config.pad_token_id = self.tokenizer.pad_token_id

        self.train_dataset, self.eval_dataset = self.load_and_tokenize_data()

        self.data_collator = DataCollatorForLanguageModeling(
            tokenizer=self.tokenizer,
            mlm=False,
        )

        self.training_args = TrainingArguments(
            output_dir=self.output_dir,
            overwrite_output_dir=True,
            num_train_epochs=self.epochs,
            per_device_train_batch_size=self.batch_size,
            gradient_accumulation_steps=self.gradient_accumulation_steps,
            learning_rate=self.learning_rate,
            save_steps=500,
            save_total_limit=1,
            eval_steps=100,
            logging_steps=20,
            logging_first_step=True,
            prediction_loss_only=True,
            disable_tqdm=False,
            report_to="none",
            fp16=torch.cuda.is_available(),
            dataloader_num_workers=2,
            push_to_hub=False,
        )

        self.trainer = Trainer(
            model=self.model,
            args=self.training_args,
            train_dataset=self.train_dataset,
            eval_dataset=self.eval_dataset,
            data_collator=self.data_collator,
            tokenizer=self.tokenizer,
        )

    def load_and_tokenize_data(self):
        with open(self.dataset_path, encoding="utf-8") as f:
            lines = [line.strip() for line in f if line.strip()]

        train_lines, eval_lines = train_test_split(lines, test_size=self.test_size, random_state=42)

        train_dataset = Dataset.from_list([{"text": l} for l in train_lines])
        eval_dataset = Dataset.from_list([{"text": l} for l in eval_lines])

        def tokenize(example):
            return self.tokenizer(example["text"], truncation=True, max_length=self.max_length)

        return train_dataset.map(tokenize), eval_dataset.map(tokenize)

    def train(self):
        self.trainer.train()

    def save(self):
        self.model.save_pretrained(self.output_dir)
        self.tokenizer.save_pretrained(self.output_dir)

/Users/danilkladnitsky/.pyenv/versions/3.10.4/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
trainer = ChineseGPTTrainer(
    model_name="uer/gpt2-chinese-cluecorpussmall",
    dataset_path=DATASET_PATH + "hsk2_labeled.txt",
    output_dir=SAVE_MODEL_PATH + "hsk2-gpt2"
)

trainer.train()
trainer.save()

Map: 100%|██████████| 690/690 [00:00<00:00, 15230.82 examples/s]
/var/folders/p9/gl1s91rn2fv_77662_261gb80000gn/T/ipykernel_13281/1488441287.py:67: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  self.trainer = Trainer(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
1,4.763900
20,2.201400
40,1.269700
60,1.174000
80,1.099000
100,1.077700
120,1.074800
140,1.049400
160,1.079800
180,1.052100


In [21]:
import copy

class HyperparamSearch:
    def __init__(self, base_config):
        self.base_config = base_config
        self.results = []

    def search(self, param_grid):
        """
        param_grid: dict of parameter lists, e.g.
        {
            "batch_size": [2, 4],
            "learning_rate": [5e-5, 2e-5],
            "epochs": [3, 5]
        }
        """
        import itertools
        keys = list(param_grid.keys())
        combinations = list(itertools.product(*param_grid.values()))

        for i, combo in enumerate(combinations):
            print(f"\n🚀 Running config {i+1}/{len(combinations)}")
            config = copy.deepcopy(self.base_config)
            for k, v in zip(keys, combo):
                setattr(config, k, v)

            trainer = ChineseGPTTrainer(
                model_name=config.model_name,
                dataset_path=config.dataset_path,
                output_dir=os.path.join(config.output_dir, f"run_{i}"),
                test_size=config.test_size,
                max_length=config.max_length,
                batch_size=config.batch_size,
                gradient_accumulation_steps=config.gradient_accumulation_steps,
                epochs=config.epochs,
                learning_rate=config.learning_rate
            )

            trainer.train()
            metrics = trainer.trainer.evaluate()
            loss = metrics.get("eval_loss", float("inf"))

            self.results.append({
                "run": i,
                "config": combo,
                "loss": loss
            })

        self.results.sort(key=lambda x: x["loss"])
        return self.results[0]  # return best config

In [ ]:
from types import SimpleNamespace

base_config = SimpleNamespace(
    model_name="uer/gpt2-chinese-cluecorpussmall",
    dataset_path=DATASET_PATH + "hsk1_labeled.txt",
    output_dir=SAVE_MODEL_PATH + "grid_search/",
    test_size=0.1,
    max_length=64,
    batch_size=2,
    gradient_accumulation_steps=4,
    epochs=3,
    learning_rate=2e-5
)

param_grid = {
    "batch_size": [2, 4],
    "learning_rate": [2e-5, 5e-5],
    "epochs": [3, 5]
}

searcher = HyperparamSearch(base_config)
best = searcher.search(param_grid)

print("\n✅ Best configuration:")
print(best)


🚀 Running config 1/8


Map: 100%|██████████| 185/185 [00:00<00:00, 14648.79 examples/s]
/var/folders/p9/gl1s91rn2fv_77662_261gb80000gn/T/ipykernel_13281/1488441287.py:67: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  self.trainer = Trainer(


Step,Training Loss
1,4.821800
20,2.055500
40,1.065800
60,0.988700
80,0.934300
100,0.914900
120,0.892800
140,0.873500
160,0.879000
180,0.856500


In [17]:
from transformers import GPT2LMHeadModel, AutoTokenizer
import torch
import re

class ChineseSentenceGenerator:
    def __init__(self, model_path: str, device: str = None):
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        self.model = GPT2LMHeadModel.from_pretrained(model_path)

        self.model.eval()
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)

        # Ensure pad token is defined
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
            self.model.config.pad_token_id = self.tokenizer.pad_token_id

    def generate(self, word: str, max_length: int = 60, num_return_sequences: int = 1) -> list:
        prompt = f"输入词语：{word}。生成句子:"
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.device)

        with torch.no_grad():
            outputs = self.model.generate(
                input_ids=inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                max_length=max_length,
                do_sample=True,
                top_k=50,
                top_p=0.95,
                temperature=0.5,
                num_return_sequences=num_return_sequences,
                pad_token_id=self.model.config.pad_token_id,
                eos_token_id=self.tokenizer.eos_token_id,
                repetition_penalty=1.2
            )

        results = []
        for output in outputs:
            decoded = self.tokenizer.decode(output, skip_special_tokens=True)
            sentence = decoded.replace(prompt, "").replace(" ", "").strip()
            print(sentence)
            sentence = re.split(r"[。！？]", sentence)[0] + "。"  # stop at first punctuation
            results.append(sentence)

        return results

In [20]:
generator = ChineseSentenceGenerator(SAVE_MODEL_PATH + "hsk2-gpt2")

results = generator.generate("一起", num_return_sequences=1)
for i, sent in enumerate(results):
    print(f"📝 Sentence {i+1}: {sent}")

输入词语：一起。生成句子:！我们一起去。...，谢谢！。。。。。。。。。。。。。。
📝 Sentence 1: 输入词语：一起。
